In [82]:
# Load necessary libraries
if (!requireNamespace("readxl", quietly = TRUE)) install.packages("readxl")
if (!requireNamespace("googlesheets4", quietly = TRUE)) install.packages("googlesheets4") # nolint: line_length_linter.

library(readxl)
library(googlesheets4)


In [83]:
# remotes::install_github("glmmTMB/glmmTMB", subdir = "glmmTMB")


In [84]:
# Define input and output paths
input_path <- "https://docs.google.com/spreadsheets/d/1m3eEaxWT4Unb8jBZWKjiLivavfA0x3PT1F1Rz1eXwVE/edit?gid=0" # nolint
output_path <- "https://docs.google.com/spreadsheets/d/1RzC3DfKNUwYA-qfUN0i53hUwRkWGMBOCoYQqdqG4MQY/edit?gid=0" # nolint

local_input_path <- "D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\RegressionDF_forR.csv" # nolint

regression_df <- read.csv(local_input_path)
print(regression_df)


          date       regiao mental_health_visits total_atd periodo
1   2018-01-01       Brasil               280906  16650688       1
2   2018-02-01       Brasil               268180  15280725       2
3   2018-03-01       Brasil               315705  18712756       3
4   2018-04-01       Brasil               321409  18818333       4
5   2018-05-01       Brasil               325042  19291195       5
6   2018-06-01       Brasil               318899  17535895       6
7   2018-07-01       Brasil               334235  18796956       7
8   2018-08-01       Brasil               387238  21295471       8
9   2018-09-01       Brasil               371794  20066133       9
10  2018-10-01       Brasil               454198  24146684      10
11  2018-11-01       Brasil               349077  18438761      11
12  2018-12-01       Brasil               263849  14059001      12
13  2019-01-01       Brasil               340011  19409656      13
14  2019-02-01       Brasil               369303  20389977    

# Imports

In [85]:
# install.packages(c(
#    "lme4",
#     "performance",
#      "DHARMa",
#      "ggplot2",
#      "ggeffects",
#      "purrr",
#      "emmeans",
#      "performance",
#      "dplyr",
#      "patchwork",
#      "see",
#      "broom.mixed",
#      "dotwhisker",
#      "tidyr",
#      "stringr",
#      "glarma",
#      "brms",
#      "remotes",
#      "tibble"))

# install.packages(c("gamlss.add"))
# install.packages(c(
#    # "lubridate",
#    # "ggplot2",
#    # "purrr",
#    #"glmmTMB"
# ))
library(gamlss.add)
library(gamlss)
library(glarma)
library(dplyr)
library(MASS)
library(nlme)
library(purrr)
library(brms)
library(cmdstanr)
library(glmmTMB) # For NB GLMM with autocorrelation
# library(lme4) # Alternative for basic GLMM
# library(performance) # For model checks
# library(DHARMa) # For residual diagnostics
# library(ggplot2) # For plots
# library(ggeffects)
# library(purrr)
# library(emmeans)
# library(performance)

# library(patchwork)
# library(see)
# library(broom.mixed)
# library(dotwhisker)
# library(tidyr)
# library(stringr)
# library(glarma)
# library(brms)
# library(remotes)
# library(tibble)


In [86]:
options(repr.plot.width = 16, repr.plot.height = 8) # Para Jupyter Notebook/RMarkdown


# Model Defs

In [87]:
#' Fit GLMM with PQL estimation and AR(1) structure
#'
#' @param data Dataframe containing all variables
#' @param response_var Name of response variable (count data)
#' @param fixed_effects Vector of fixed effect variable names
#' @param offset_var Name of offset variable (should be log-transformed)
#' @param time_var Name of time variable for AR(1) structure
#' @param group_var Name of grouping variable (default intercept-only)
#' @param theta Starting value for negative binomial theta (NULL for estimation)
#'
#' @return Fitted model object
#' @export
#'
#' @examples
#' model <- fit_glmm_ar1(
#'   data = my_data,
#'   response_var = "mental_health_visits",
#'   fixed_effects = c("periodo", "Pandemia_Step", "Pandemia_Trend"),
#'   offset_var = "log_total_visits",
#'   time_var = "month"
#' )
fit_glmm_ar1 <- function(data,
                         response_var,
                         fixed_effects,
                         offset_var = NULL,
                         time_var,
                         group_var = NULL,
                         theta = 1) {
  # Load required packages
  require(nlme)
  require(MASS)

  # Create formula components
  fixed_part <- paste(
    response_var, "~",
    paste(fixed_effects, collapse = " + ")
  )

  if (!is.null(offset_var)) {
    fixed_part <- paste(fixed_part, "+ offset(", offset_var, ")")
  }

  # Handle grouping structure
  if (is.null(group_var)) {
    data$..group.. <- factor(1) # Intercept-only random effect
    random_formula <- ~ 1 | ..group..
    cor_formula <- as.formula(paste("~", time_var, "| ..group.."))
  } else {
    random_formula <- as.formula(paste("~ 1 |", group_var))
    cor_formula <- as.formula(paste("~", time_var, "|", group_var))
  }

  # Fit model
  model <- glmmPQL(
    fixed = as.formula(fixed_part),
    random = random_formula,
    family = negative.binomial(theta = theta), # quasipoisson(),#poisson(),#
    correlation = corAR1(form = cor_formula),
    data = data,
    verbose = FALSE
  )


  # Add model components to object for easier reference
  model$call$fixed <- fixed_part
  model$call$random <- random_formula
  model$call$correlation <- cor_formula

  return(model)
}


In [88]:
fit_brms_arma <- function(data,
                          response_var,
                          fixed_effects,
                          offset_var = NULL,
                          time_var,
                          group_var = NULL,
                          ar_order = 1,
                          ma_order = 0,
                          chains = 4,
                          cores = 4,
                          iter = 4000) {
  require(brms)
  require(dplyr)

  safe_cores <- max(1, parallel::detectCores() - 1) # Garante pelo menos 1 núcleo

  options(mc.cores = safe_cores) # Para Stan/brms
  # 1. Preparar a fórmula fixa
  fixed_part <- paste(response_var, "~", paste(fixed_effects, collapse = " + "))

  # 2. Adicionar offset (se existir)
  if (!is.null(offset_var)) {
    fixed_part <- paste(fixed_part, "+ offset(", offset_var, ")")
  }

  # 3. Estrutura aleatória
  if (is.null(group_var)) {
    random_formula <- "1" # Intercepto global
  } else {
    random_formula <- paste0("(1 | ", group_var, ")")
  }

  # 4. Estrutura ARMA - CORREÇÃO AQUI
  arma_parts <- c()
  if (ar_order > 0) {
    arma_parts <- c(
      arma_parts,
      paste0(
        "ar(p = ", ar_order,
        ", time = ", time_var,
        if (!is.null(group_var)) paste0(", gr = ", group_var) else "",
        ")"
      )
    )
  }
  if (ma_order > 0) {
    arma_parts <- c(
      arma_parts,
      paste0(
        "ma(q = ", ma_order,
        ", time = ", time_var,
        if (!is.null(group_var)) paste0(", gr = ", group_var) else "",
        ")"
      )
    )
  }

  # Combinar todas as partes da fórmula
  full_formula <- paste(c(fixed_part, random_formula, arma_parts), collapse = " + ")

  # 5. Ajustar modelo
  model <- brm(
    formula = as.formula(full_formula),
    family = negbinomial(), # Negative Binomial
    data = data,
    chains = safe_cores,
    cores = safe_cores,
    iter = iter,
    control = list(
      adapt_delta = 0.99, # Padrão = 0.8 (aumente para evitar divergências)
      max_treedepth = 15 # Padrão = 10
    ),
    backend = "cmdstanr"
  )

  return(model)
}


In [89]:
fit_glmm_arma <- function(data,
                          response_var,
                          fixed_effects,
                          offset_var = NULL,
                          time_var,
                          group_var = NULL,
                          theta = 1,
                          ar_order = 1, # Ordem AR(p)
                          ma_order = 0) { # Ordem MA(q)

  require(nlme)
  require(MASS)
  require(dplyr)

  # Criar fórmula fixa
  fixed_part <- paste(response_var, "~", paste(fixed_effects, collapse = " + "))

  if (!is.null(offset_var)) {
    fixed_part <- paste(fixed_part, "+ offset(", offset_var, ")")
  }

  # Estrutura aleatória
  if (is.null(group_var)) {
    data$..group.. <- factor(1)
    random_formula <- ~ 1 | ..group..
    cor_formula <- as.formula(paste("~", time_var, "| ..group.."))
  } else {
    random_formula <- as.formula(paste("~ 1 |", group_var))
    cor_formula <- as.formula(paste("~", time_var, "|", group_var))
  }

  # Estrutura de correlação ARMA
  if (ar_order > 0 | ma_order > 0) {
    cor_struct <- corARMA(form = cor_formula, p = ar_order, q = ma_order)
  } else {
    cor_struct <- NULL
  }

  # Ajustar modelo
  model <- glmmPQL(
    fixed = as.formula(fixed_part),
    random = random_formula,
    family = negative.binomial(theta = theta),
    correlation = cor_struct,
    data = data,
    verbose = FALSE
  )

  return(model)
}


In [90]:
fit_glmmTMB_ar1 <- function(data,
                            response_var,
                            fixed_effects,
                            offset_var = NULL,
                            time_var, # apenas o tempo numérico (ex: "periodo")
                            group_var = NULL,
                            family = "nbinom2") {
  require(glmmTMB)
  require(dplyr)

  # Criar variável fator de tempo internamente
  fac_name <- paste0(time_var, "_fac")
  data <- data %>%
    mutate(
      !!fac_name := factor(!!sym(time_var))
    )

  # Construir parte fixa da fórmula
  fixed_part <- paste(response_var, "~", paste(fixed_effects, collapse = " + "))
  if (!is.null(offset_var)) {
    fixed_part <- paste(fixed_part, "+ offset(", offset_var, ")")
  }

  # Definir grupo
  grupo <- ifelse(is.null(group_var), "1", group_var)

  # Estrutura AR(1)
  ar1_struct <- paste0("ar1(", fac_name, " + 0 | ", grupo, ")")

  # Fórmula completa
  full_formula <- as.formula(paste(fixed_part, "+", ar1_struct))

  # Definir família
  fam <- switch(family,
    "nbinom2" = nbinom2(),
    "nbinom1" = nbinom1(),
    "poisson" = poisson(),
    stop("Família não suportada.")
  )

  # Ajustar modelo
  model <- glmmTMB(
    formula = full_formula,
    data = data,
    family = fam
  )

  return(model)
}


In [91]:
fit_glarma <- function(data,
                       response_var,
                       fixed_effects,
                       offset_var = NULL,
                       time_var,
                       ar_order = 1,
                       ma_order = 0,
                       family = "poisson") {
  require(glarma)
  require(dplyr)

  # Ordenar os dados pelo tempo
  data <- data %>% arrange(!!sym(time_var))

  # Matriz de design
  formula_fixed <- as.formula(paste("~", paste(fixed_effects, collapse = " + ")))
  X <- model.matrix(formula_fixed, data = data)

  # Variável resposta
  y <- data[[response_var]]

  # Offset
  if (!is.null(offset_var)) {
    offset <- as.numeric(data[[offset_var]])
  } else {
    offset <- rep(0, length(y)) # glarma espera offset explícito, mesmo que nulo
  }

  # Tipo de família
  fam <- switch(family,
    "poisson" = "Poi",
    "nbinom" = "NegBin",
    stop("Família não suportada.")
  )

  # Ajuste do modelo GLARMA
  model <- glarma(
    y = y,
    X = X,
    offset = offset,
    type = fam,
    phiLags = if (ar_order > 0) 1:ar_order else NULL,
    thetaLags = if (ma_order > 0) 1:ma_order else NULL,
    method = "FS",
    residuals = TRUE
  )

  return(model)
}


In [92]:
fit_gamlss_arma <- function(data,
                            response_var,
                            fixed_effects,
                            random_effects = NULL,
                            offset_var = NULL,
                            time_var,
                            ar_order = 1,
                            ma_order = 0,
                            family = "PO",
                            correlation = TRUE,
                            ...) {
  require(gamlss)
  require(gamlss.add)
  require(dplyr)

  # Ordenar os dados pelo tempo
  data <- data %>% arrange(!!sym(time_var))

  # Criar fórmula para efeitos fixos
  formula_fixed <- paste(response_var, "~", paste(fixed_effects, collapse = " + "))

  # Adicionar efeitos aleatórios se especificados
  if (!is.null(random_effects)) {
    formula_fixed <- paste(formula_fixed, "+", paste0("random(", random_effects, ")", collapse = " + "))
  }

  # Adicionar offset se especificado
  if (!is.null(offset_var)) {
    formula_fixed <- paste(formula_fixed, "+ offset(", offset_var, ")")
  }

  # Converter para fórmula
  formula_fixed <- as.formula(formula_fixed)

  # Ajuste do modelo GAMLSS com ARMA
  model <- gamlss(
    formula = formula_fixed,
    family = family,
    data = data,
    trace = FALSE,
    # Adicionar controle ARMA através do parâmetro 'control'
    control = gamlss.control(
      n.cyc = 100,
      c.crit = 0.01,
      mu.step = 0.1,
      sigma.step = 0.1,
      nu.step = 0.1,
      tau.step = 0.1
    ),
    # Especificar a estrutura de correlação ARMA
    correlation = if (correlation) corARMA(p = ar_order, q = ma_order) else NULL,
    ...
  )

  return(model)
}


# Model Run

In [ ]:
# Inicialize uma lista para guardar os modelos
models_list <- list()

# Loop para ajustar os modelos por região e armazenar na lista
for (reg in unique(regression_df$regiao)) {
  require(dplyr)
  cat("=============================================Rodando:", reg, "\n")
  reg_data <- regression_df %>% filter(regiao == reg)

  # model <- fit_brms_arma(
  #  data = reg_data,
  #  response_var = "mental_health_visits",
  #  fixed_effects = c(
  #    "periodo", "Pandemia_Step", "Pandemia_Trend",
  #    "PosPandemia_Step", "PosPandemia_Trend" # , "cos1", "sin1"
  #  ),
  #  offset_var = "offset",
  #  time_var = "periodo",
  #  ar_order = 1,
  #  ma_order = 0,
  # )
  # model <- fit_glmmTMB_ar1(
  # data = reg_data,
  # response_var = "mental_health_visits",
  # fixed_effects = c(
  #   "periodo", "Pandemia_Step", "Pandemia_Trend",
  #   "PosPandemia_Step", "PosPandemia_Trend"  , "cos1", "sin1"
  # ),
  # offset_var = "offset",
  # time_var = "periodo"
  # )
  # model <- fit_glarma(
  #  data = reg_data,
  #  response_var = "mental_health_visits",
  #  fixed_effects = c(
  #    "periodo", "Pandemia_Step", "Pandemia_Trend",
  #    "PosPandemia_Step", "PosPandemia_Trend" # , "cos1", "sin1"
  #  ),
  #  offset_var = "offset",
  #  time_var = "periodo",
  #  ar_order = 1,
  #  ma_order = 1,
  #  family = "poisson"
  # )
  model <- fit_gamlss_arma(
    data = reg_data,
    response_var = "mental_health_visits",
    fixed_effects = c(
      "periodo", "Pandemia_Step", "Pandemia_Trend",
      "PosPandemia_Step", "PosPandemia_Trend" # , "cos1", "sin1"
    ),
    offset_var = "offset",
    time_var = "periodo",
    ar_order = 1,
    ma_order = 1,
    family = "NBI"
  )

  # Armazenar o modelo na lista com a chave sendo a região
  models_list[[reg]] <- model
  # print('-----R² condicional/marginal')
  # print(performance::r2(model))  # R² condicional/marginal
  # Imprimir o resumo do modelo para verificação
  print(summary(model))
}


=============================================Rodando: Brasil 
 Family: nbinom2  ( log )
Formula:          
mental_health_visits ~ periodo + Pandemia_Step + Pandemia_Trend +  
    PosPandemia_Step + PosPandemia_Trend + cos1 + sin1 + offset(offset) +  
    ar1(periodo_fac + 0 | 1)
Data: data

     AIC      BIC   logLik deviance df.resid 
  1684.8   1705.3   -833.4   1666.8       63 


Dispersion parameter for nbinom2 family ():  382 

Conditional model:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)        2.794248   0.020720  134.86  < 2e-16 ***
periodo            0.011743   0.001310    8.97  < 2e-16 ***
Pandemia_Step      0.168200   0.030184    5.57 2.51e-08 ***
Pandemia_Trend    -0.015325   0.001955   -7.84 4.55e-15 ***
PosPandemia_Step  -0.191611   0.054382   -3.52 0.000426 ***
PosPandemia_Trend  0.001143   0.002315    0.49 0.621427    
cos1              -0.009248   0.008357   -1.11 0.268452    
sin1              -0.008871   0.009412   -0.94 0.345927    
---


# Diagnóstico

In [96]:
# GLMM TMB
if (!dir.exists("acf_plots")) dir.create("acf_plots")
if (!dir.exists("model_tests")) dir.create("model_tests")

sink(file = "model_tests/ljung_box_results.txt")

for (reg in names(models_list)) {
  model <- models_list[[reg]]

  # Resíduos de Pearson
  res <- residuals(model, type = "pearson")

  # Plot ACF/PACF
  png(
    filename = paste0("acf_plots/", reg, "_acf_pacf.png"),
    width = 10, height = 5, units = "in", res = 300
  )
  par(mfrow = c(1, 2))

  acf(res, main = paste("ACF -", reg), lag.max = 36, ylim = c(-0.5, 1))
  pacf(res, main = paste("PACF -", reg), lag.max = 36, ylim = c(-0.5, 1))

  dev.off()

  # Teste Ljung-Box
  cat("\n----------------------------------------\n")
  cat("Região:", reg, "\n")
  cat("Testes Ljung-Box para resíduos de Pearson:\n\n")

  for (lag in c(1, 9, 12, 24, 25)) {
    test <- Box.test(res, lag = lag, type = "Ljung-Box")
    cat(sprintf("Lag %2d: p-valor = %.4f", lag, test$p.value))
    if (test$p.value < 0.05) {
      cat(" *")
      if (test$p.value < 0.01) cat("*")
      if (test$p.value < 0.001) cat("*")
    }
    cat("\n")
  }

  cat("Gráficos e testes gerados para:", reg, "\n")
}

sink()


In [95]:
# Criar pastas para salvar os resultados
if (!dir.exists("acf_plots")) {
  dir.create("acf_plots")
}
if (!dir.exists("model_tests")) {
  dir.create("model_tests")
}

# Arquivo para salvar os testes Ljung-Box
sink(file = "model_tests/ljung_box_results.txt")

# Loop através de cada modelo na lista
for (reg in names(models_list)) {
  model <- models_list[[reg]]

  # Extrair resíduos normalizados
  res <- residuals(model, type = "normalized")

  # Gráficos ACF/PACF
  png(
    filename = paste0("acf_plots/", reg, "_acf_pacf.png"),
    width = 10, height = 5, units = "in", res = 300
  )
  par(mfrow = c(1, 2))

  # Plot ACF
  acf(res,
    main = paste("ACF -", reg),
    lag.max = 36,
    ylim = c(-0.5, 1)
  )

  # Plot PACF
  pacf(res,
    main = paste("PACF -", reg),
    lag.max = 36,
    ylim = c(-0.5, 1)
  )

  dev.off()

  # Teste Ljung-Box para diferentes lags
  cat("\n----------------------------------------\n")
  cat("Região:", reg, "\n")
  cat("Testes Ljung-Box para resíduos:\n\n")

  # Testar para lags específicos (1, 9, 12, 24, 25)
  for (lag in c(1, 9, 12, 24, 25)) {
    test <- Box.test(res, lag = lag, type = "Ljung-Box")
    cat(sprintf("Lag %2d: p-valor = %.4f", lag, test$p.value))

    # Adicionar asterisco para significância
    if (test$p.value < 0.05) {
      cat(" *")
      if (test$p.value < 0.01) cat("*")
      if (test$p.value < 0.001) cat("*")
    }
    cat("\n")
  }

  # Mostrar mensagem de progresso
  cat("Gráficos e testes gerados para:", reg, "\n")
}

# Fechar o arquivo de saída
sink()


ERROR: Error in match.arg(type): 'arg' deve ser um dentre "response", "pearson", "working"


# Plot

In [ ]:
library(ggplot2)
library(dplyr)

for (reg in names(models_list)) {
  cat("Plotando:", reg, "\n")

  model <- models_list[[reg]]
  reg_data <- regression_df %>% filter(regiao == reg)
  reg_data$fitted <- fitted(model)

  coefs <- model$coefficients$fixed

  intercept <- coefs["(Intercept)"]
  b_time <- coefs["periodo"]
  b_pstep <- coefs["Pandemia_Step"]
  b_ptrend <- coefs["Pandemia_Trend"]
  b_poststep <- coefs["PosPandemia_Step"]
  b_posttrend <- coefs["PosPandemia_Trend"]

  reg_data <- reg_data %>%
    mutate(
      fase = case_when(
        PosPandemia_Step == 1 ~ "pos",
        Pandemia_Step == 1 ~ "pandemia",
        TRUE ~ "pre"
      ),
      predicted_segmented = case_when(
        fase == "pre" ~ exp(intercept + b_time * periodo),
        fase == "pandemia" ~ exp(intercept + b_pstep + b_time * periodo + b_ptrend * Pandemia_Trend),
        # fase == "pos" ~ exp(intercept + b_pstep + b_poststep + b_time * periodo +
        #                      b_ptrend * Pandemia_Trend + b_posttrend * PosPandemia_Trend)
        fase == "pos" ~ exp(intercept + b_poststep + b_time * periodo +
          +b_posttrend * PosPandemia_Trend)
      ),
      predicted_counterfactual = exp(intercept + b_time * periodo)
    )

  p <- ggplot(reg_data, aes(x = periodo)) +
    geom_point(aes(y = IR), color = "black", size = 2) +
    geom_line(aes(y = predicted_segmented), color = "#8c00ff", size = 1) +
    geom_line(aes(y = predicted_counterfactual), color = "#000000", linetype = "dashed", size = 1) +
    labs(
      title = paste("ITS com contrafactual -", reg),
      y = "Taxa por 1000 atendimentos",
      x = "Período"
    ) +
    theme_minimal()

  print(p)
}


Plotando: Brasil 


ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `predicted_segmented = case_when(...)`.
[1mCaused by error in `case_when()`:[22m
[1m[22m[33m![39m Failed to evaluate the right-hand side of formula 3.
[1mCaused by error in `+b_posttrend`:[22m
[33m![39m argumento inválido para operador unário
